In [3]:
import numpy as np
import tensorflow as tf

actions = [
    'apron',
    'cold',
    'connect',
    'good',
    'hot',
    'more',
    'when'
    ]

data = np.concatenate([
    np.load("seq_apron_1726998454.npy"),
    np.load("seq_cold_1726998454.npy"),
    np.load("seq_connect_1726998454.npy"),
    np.load("seq_good_1726998454.npy"),
    np.load("seq_hot_1726998454.npy"),
    np.load("seq_more_1726998454.npy"),
    np.load("seq_when_1726998454.npy")
])

In [ ]:
x_data = data [:, :, :-1]
labels = data [:, 0, -1]

x_data.shape, labels.shape

((15764, 30, 99), (15764,))

In [ ]:
#확인용
import numpy as np

print("x_data shape:", x_data.shape)
print("labels shape:", labels.shape)

# 첫 번째 샘플의 첫 번째 타임스텝의 랜드마크 좌표
print("첫 번째 샘플의 첫 번째 타임스텝의 랜드마크 좌표:\n", x_data[0, 0, :])

# 첫 번째 샘플의 레이블
print("첫 번째 샘플의 레이블:", labels[0])

# 여러 샘플의 랜드마크 좌표와 레이블
for i in range(5):  # 예시로 5개의 샘플\
    print(f"\n샘플 {i+1}:")
    print("랜드마크 좌표:\n", x_data[i, :, :])
    print("레이블:", labels[i])


x_data shape: (15764, 30, 99)
labels shape: (15764,)
첫 번째 샘플의 첫 번째 타임스텝의 랜드마크 좌표:
 [ 6.69020832e-01  8.91961753e-01  2.93402138e-07  0.00000000e+00
  6.59965456e-01  8.23270261e-01 -1.06393537e-02  0.00000000e+00
  6.09507740e-01  7.71163583e-01 -1.56866424e-02  0.00000000e+00
  5.66832423e-01  7.46440411e-01 -1.91925280e-02  0.00000000e+00
  5.38931787e-01  7.23103046e-01 -2.41311174e-02  0.00000000e+00
  5.71766913e-01  7.78887689e-01 -2.73854267e-02  0.00000000e+00
  5.10803938e-01  7.61017442e-01 -4.31685708e-02  0.00000000e+00
  4.65506136e-01  7.53170669e-01 -5.36561310e-02  0.00000000e+00
  4.29650992e-01  7.49847531e-01 -6.10543303e-02  0.00000000e+00
  5.62385261e-01  8.24731469e-01 -3.11554205e-02  0.00000000e+00
  4.96023923e-01  8.05471718e-01 -4.66868803e-02  0.00000000e+00
  4.48400408e-01  7.94400275e-01 -5.72938025e-02  0.00000000e+00
  4.08606559e-01  7.84470975e-01 -6.49539903e-02  0.00000000e+00
  5.58715105e-01  8.73368025e-01 -3.47791426e-02  0.00000000e+00
  4.960

In [ ]:
from tensorflow.keras.utils import to_categorical

y_data = to_categorical(labels, num_classes =len(actions))
y_data.shape

(15764, 7)

In [ ]:
from sklearn.model_selection import train_test_split

x_data = x_data.astype(np.float32)
y_data = y_data.astype(np.float32)

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2021)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)


(14187, 30, 99) (14187, 7)
(1577, 30, 99) (1577, 7)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

def GRU_model(learning_rate=0.001, beta_1=0.9, beta_2=0.999):
    model = Sequential()
    model.add(GRU(units=50, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2]), activation='tanh'))
    model.add(GRU(units=50, activation='tanh'))
    model.add(Dense(units=7, activation = "softmax"))

    # 옵티마이저 정의
    optimizer = Adam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2, epsilon=1e-07)

    # 모델 컴파일 (회귀 문제인 경우, loss를 'mean_squared_error'로 설정)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# 모델 생성
model = GRU_model()

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# 모델 학습
history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=200,
    callbacks=[
        # val_loss를 기준으로 최적 모델 저장
        ModelCheckpoint('models/model.keras', monitor='val_loss', verbose=1, save_best_only=True, mode='auto'),
        # val_loss가 개선되지 않을 때 learning rate 감소
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=50, verbose=1, mode='auto')
    ]
)




Epoch 1/200
444/444 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8866 - loss: 0.3967
Epoch 1: val_loss improved from inf to 0.17122, saving model to models/model.keras
444/444 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.8868 - loss: 0.3961 - val_accuracy: 0.9505 - val_loss: 0.1712 - learning_rate: 0.0010
Epoch 2/200
435/444 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9912 - loss: 0.0362
Epoch 2: val_loss improved from 0.17122 to 0.00182, saving model to models/model.keras
444/444 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9913 - loss: 0.0357 - val_accuracy: 1.0000 - val_loss: 0.0018 - learning_rate: 0.0010
Epoch 3/200
443/444 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9992 - loss: 0.0043
Epoch 3: val_loss did not improve from 0.00182
444/444 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9992 - loss: 0.0044 - val_accuracy: 1.0000 - val_loss: 0.0020 - learning_rate: 0.0010
Epoch 4/200
439/444 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9971 - loss: 0.0094
Epoch 4: val

In [ ]:
# 탐색할 하이퍼파라미터 값들을 정의합니다.
param_grid = {
    'learning_rate': [0.1, 0.01, 0.001],
    'beta_1': [0.8, 0.9, 0.95],
    'beta_2': [0.99, 0.995, 0.999]
}

# 그리드 탐색을 수행합니다.
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1)
grid_result = grid.fit(x_train, y_train)

# 결과를 출력합니다.
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
